# Import Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import os
import folium
import json

In [2]:
%matplotlib inline

In [3]:
# Import ".json" file for the U.S. 
country_geo = r'/Users/mimielaurant/Documents/Career Foundry/Immersion_Achievement 6/Citibike Project/02 Data/Original Data/new-york-city-boroughs-ny_.geojson'

In [4]:
# define path
path = '/Users/mimielaurant/Documents/Career Foundry/Immersion_Achievement 6/Citibike Project'

In [5]:
# import citibike_clean.csv
data = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'citibikes_new.csv'), index_col = False)

In [6]:
data.head()

,Unnamed: 0,trip_id,bike_id,weekday,start_hour,start_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,...,end_station_name,end_station_latitude,end_station_longitude,trip_duration,subscriber,birth_year,gender,age,Trip Range,trip_minutes
0,0,LnQzQk,16013,Mon,18,2013-09-09 18:18:55,523,W 38 St & 8 Ave,40.754666,-73.991382,...,W 20 St & 7 Ave,40.742388,-73.997262,993,Subscriber,1968-01-01,2,45.692282,long trip,16
1,1,IL9boN,15230,Thu,18,2013-09-12 18:38:53,257,Lispenard St & Broadway,40.719392,-74.002472,...,St Marks Pl & 2 Ave,40.728419,-73.987140,581,Subscriber,1983-01-01,1,30.699541,average trip,9
2,2,46clGB,17942,Wed,19,2013-09-18 19:44:04,479,9 Ave & W 45 St,40.760193,-73.991255,...,W 56 St & 10 Ave,40.768254,-73.988639,361,Subscriber,1989-01-01,1,24.714600,average trip,6
3,3,v7vdFt,19683,Sat,11,2013-09-28 11:54:37,527,E 33 St & 1 Ave,40.743156,-73.974347,...,E 52 St & 2 Ave,40.756014,-73.967416,561,Subscriber,1988-01-01,2,25.743160,average trip,9
4,4,VGBsb5,18024,Sat,18,2013-09-07 18:08:22,521,8 Ave & W 31 St,40.750450,-73.994811,...,E 31 St & 3 Ave,40.743943,-73.979661,2296,Non-Subscriber,1978-01-01,0,35.685211,long trip,38


In [7]:
data.shape

(49977, 22)

In [8]:
data['station_count'] = data.groupby('start_station_name')['start_station_name'].transform('count')

In [9]:
data.head()

,Unnamed: 0,trip_id,bike_id,weekday,start_hour,start_time,start_station_id,start_station_name,start_station_latitude,start_station_longitude,...,end_station_latitude,end_station_longitude,trip_duration,subscriber,birth_year,gender,age,Trip Range,trip_minutes,station_count
0,0,LnQzQk,16013,Mon,18,2013-09-09 18:18:55,523,W 38 St & 8 Ave,40.754666,-73.991382,...,40.742388,-73.997262,993,Subscriber,1968-01-01,2,45.692282,long trip,16,228
1,1,IL9boN,15230,Thu,18,2013-09-12 18:38:53,257,Lispenard St & Broadway,40.719392,-74.002472,...,40.728419,-73.987140,581,Subscriber,1983-01-01,1,30.699541,average trip,9,118
2,2,46clGB,17942,Wed,19,2013-09-18 19:44:04,479,9 Ave & W 45 St,40.760193,-73.991255,...,40.768254,-73.988639,361,Subscriber,1989-01-01,1,24.714600,average trip,6,226
3,3,v7vdFt,19683,Sat,11,2013-09-28 11:54:37,527,E 33 St & 1 Ave,40.743156,-73.974347,...,40.756014,-73.967416,561,Subscriber,1988-01-01,2,25.743160,average trip,9,207
4,4,VGBsb5,18024,Sat,18,2013-09-07 18:08:22,521,8 Ave & W 31 St,40.750450,-73.994811,...,40.743943,-73.979661,2296,Non-Subscriber,1978-01-01,0,35.685211,long trip,38,410


In [10]:
# import Nominatim API
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [11]:
geolocator = Nominatim(user_agent="bikeshare")
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1, max_retries=0)

In [12]:
# test
reverse("40.754666, -73.991382").raw['address']

{'amenity': 'Citi Bike - W 38 St & 8 Ave',
 'road': 'West 38th Street',
 'neighbourhood': 'Manhattan Community Board 5',
 'suburb': 'Manhattan',
 'county': 'New York County',
 'city': 'City of New York',
 'state': 'New York',
 'ISO3166-2-lvl4': 'US-NY',
 'postcode': '10018',
 'country': 'United States',
 'country_code': 'us'}

# Wrangling Data

In [13]:
# create locations dataframe for map markers
locations = data[['start_station_id','start_station_name',\
'start_station_latitude','start_station_longitude', 'station_count']]\
.drop_duplicates(subset = ['start_station_id'])


In [14]:
# rename columns and reset index
locations.columns=['stationid','station name','latitude','longitude', 'count']
locations.set_index('stationid', inplace=True)
locations

,station name,latitude,longitude,count
stationid,,,,
523,W 38 St & 8 Ave,40.754666,-73.991382,228
257,Lispenard St & Broadway,40.719392,-74.002472,118
479,9 Ave & W 45 St,40.760193,-73.991255,226
527,E 33 St & 1 Ave,40.743156,-73.974347,207
521,8 Ave & W 31 St,40.750450,-73.994811,410
...,...,...,...,...
431,Hanover Pl & Livingston St,40.688646,-73.982634,14
331,Pike St & Monroe St,40.711731,-73.991930,54
443,Bedford Ave & S 9th St,40.708531,-73.964090,20


In [15]:
# use API to get detailed location information for each statation
locations_2=[]
for index, row in locations.iterrows():
    locations_2.append(reverse("{}, {}".format(row['latitude'],\
    row['longitude'])).raw['address'])
pd.DataFrame(locations_2[:10])

,amenity,road,neighbourhood,suburb,county,city,state,ISO3166-2-lvl4,postcode,country,country_code,house_number,railway,tourism
0,Citi Bike - W 38 St & 8 Ave,West 38th Street,Manhattan Community Board 5,Manhattan,New York County,City of New York,New York,US-NY,10018,United States,us,NaN,NaN,NaN
1,Citi Bike - Lispenard St & Broadway,Lispenard Street,Manhattan Community Board 1,Manhattan,New York County,City of New York,New York,US-NY,10013,United States,us,NaN,NaN,NaN
2,Citi Bike - 9th Avenue & West 45th Street,9th Avenue,Clinton,Manhattan,New York County,City of New York,New York,US-NY,10019,United States,us,NaN,NaN,NaN
3,NaN,East 33rd Street,Kips Bay,Manhattan,New York County,City of New York,New York,US-NY,10016,United States,us,377,NaN,NaN
4,NaN,8th Avenue,Manhattan Community Board 4,Manhattan,New York County,City of New York,New York,US-NY,10122,United States,us,NaN,NaN,NaN
5,NaN,East 8th Street,Manhattan Community Board 2,Manhattan,New York County,City of New York,New York,US-NY,10003,United States,us,NaN,Astor Place,NaN
6,Citi bike - W 4 St & 7 Ave S,West 4th Street,Manhattan Community Board 2,Manhattan,New York County,City of New York,New York,US-NY,10014,United States,us,NaN,NaN,NaN
7,Citi Bike - Bank St & Hudson St,Bank Street,Manhattan Community Board 2,Manhattan,New York County,City of New York,New York,US-NY,10014,United States,us,NaN,NaN,NaN
8,NaN,Rockefeller Plaza,Manhattan Community Board 5,Manhattan,New York County,City of New York,New York,US-NY,10111,United States,us,45,NaN,Rockefeller Center
9,Citi Bike - West 24th Street & 7th Avenue,West 24th Street,Manhattan Community Board 4,Manhattan,New York County,City of New York,New York,US-NY,10011,United States,us,NaN,NaN,NaN


In [16]:
# create new stations dataframe
dfstations = pd.DataFrame(index=locations.index, data=locations_2,\
      columns=['neighbourhood','suburb','postcode'])
dfstations.rename(columns={"neighbourhood":"neighborhood",\
      "suburb": "borough", "postcode": "zipcode"}\
      ,inplace=True )

In [17]:
# shorten zip codes and change data types
dfstations['neighborhood'] = dfstations['neighborhood'].astype('category')
dfstations['borough'] = dfstations['borough'].astype('category')
dfstations['zipcode'] = dfstations['zipcode'].str[:5].astype('category')
cols = ['neighborhood', 'borough']

In [18]:
# add station name, count, latitude, and longitude to dataframe
dfstations[['station_name','latitude','longitude', 'count']] =\
 locations[['station name','latitude','longitude', 'count']]

In [19]:
dfstations.head(10)

,neighborhood,borough,zipcode,station_name,latitude,longitude,count
stationid,,,,,,,
523,Manhattan Community Board 5,Manhattan,10018,W 38 St & 8 Ave,40.754666,-73.991382,228
257,Manhattan Community Board 1,Manhattan,10013,Lispenard St & Broadway,40.719392,-74.002472,118
479,Clinton,Manhattan,10019,9 Ave & W 45 St,40.760193,-73.991255,226
527,Kips Bay,Manhattan,10016,E 33 St & 1 Ave,40.743156,-73.974347,207
521,Manhattan Community Board 4,Manhattan,10122,8 Ave & W 31 St,40.750450,-73.994811,410
293,Manhattan Community Board 2,Manhattan,10003,Lafayette St & E 8 St,40.730287,-73.990765,505
380,Manhattan Community Board 2,Manhattan,10014,W 4 St & 7 Ave S,40.734011,-74.002939,386
346,Manhattan Community Board 2,Manhattan,10014,Bank St & Hudson St,40.736529,-74.006180,180
510,Manhattan Community Board 5,Manhattan,10111,W 51 St & 6 Ave,40.760660,-73.980420,171


In [20]:
# export data frame
dfstations.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'citibike_stations.csv'))

In [21]:
dfstations['station_name'] = dfstations['station_name'].astype('string')

In [22]:
dfstations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 330 entries, 523 to 321
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   neighborhood  280 non-null    category
 1   borough       330 non-null    category
 2   zipcode       320 non-null    category
 3   station_name  330 non-null    string  
 4   latitude      330 non-null    float64 
 5   longitude     330 non-null    float64 
 6   count         330 non-null    int64   
dtypes: category(3), float64(2), int64(1), string(1)
memory usage: 17.8 KB


# Generating a map using Folium

In [23]:
# Make an empty map
map = folium.Map(location=[40.712776, -74.005974], tiles="OpenStreetMap", zoom_start=12)

# Show the map
map

In [24]:
import math

# add marker one by one on the map, and account for Mercator deformation
for city in dfstations.itertuples():
   # local_deformation = math.cos(city.latitude * math.pi / 180)
    folium.Circle(
        location=[city.latitude, city.longitude],
        popup='%s (%.1f)' % (city.station_name, city.count),
       radius=city.count,
    ).add_to(map)

# Show the map again
map

In [25]:
# sort dfstations to find top 10 most frequented stations
dfstations2 = dfstations.sort_values(by='count', ascending=False)
print(dfstations2)

                          neighborhood    borough zipcode  \
stationid                                                   
459        Manhattan Community Board 4  Manhattan   10011   
293        Manhattan Community Board 2  Manhattan   10003   
519        Manhattan Community Board 5  Manhattan   10037   
497        Manhattan Community Board 5  Manhattan   10010   
426        Manhattan Community Board 1  Manhattan   10282   
...                                ...        ...     ...   
119                                NaN   Brooklyn   11251   
321                              Dumbo   Brooklyn   11201   
372                       Clinton Hill   Brooklyn   11225   
2005                               NaN   Brooklyn   11251   
2001                               NaN   Brooklyn   11251   

                            station_name   latitude  longitude  count  
stationid                                                              
459                     W 20 St & 11 Ave  40.746745 -74.007756

In [26]:
df_first_10 = dfstations2.head(10)
print(df_first_10)

                          neighborhood    borough zipcode  \
stationid                                                   
459        Manhattan Community Board 4  Manhattan   10011   
293        Manhattan Community Board 2  Manhattan   10003   
519        Manhattan Community Board 5  Manhattan   10037   
497        Manhattan Community Board 5  Manhattan   10010   
426        Manhattan Community Board 1  Manhattan   10282   
521        Manhattan Community Board 4  Manhattan   10122   
382        Manhattan Community Board 2  Manhattan   10003   
285        Manhattan Community Board 2  Manhattan   10003   
380        Manhattan Community Board 2  Manhattan   10014   
477        Manhattan Community Board 5  Manhattan   10036   

                      station_name   latitude  longitude  count  
stationid                                                        
459               W 20 St & 11 Ave  40.746745 -74.007756    536  
293          Lafayette St & E 8 St  40.730287 -73.990765    505  
519

In [27]:
# export first 10
df_first_10.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'top_10.csv'))

In [28]:
top_10 = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'top_10.csv'), index_col = False)

In [29]:
top_10.head()

,stationid,neighborhood,borough,zipcode,station_name,latitude,longitude,count
0,459,Manhattan Community Board 4,Manhattan,10011,W 20 St & 11 Ave,40.746745,-74.007756,536
1,293,Manhattan Community Board 2,Manhattan,10003,Lafayette St & E 8 St,40.730287,-73.990765,505
2,519,Manhattan Community Board 5,Manhattan,10037,Pershing Square N,40.751884,-73.977702,503
3,497,Manhattan Community Board 5,Manhattan,10010,E 17 St & Broadway,40.737050,-73.990093,496
4,426,Manhattan Community Board 1,Manhattan,10282,West St & Chambers St,40.717548,-74.013221,465


In [30]:
top_10.shape

(10, 8)

In [31]:
# Create a Folium map centered around the mean of the coordinates
map = folium.Map(location=[top_10['latitude'].mean(), top_10['longitude'].mean()], zoom_start=12)
map

In [40]:
# Add CircleMarkers for each city
for index, row in top_10.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['count'] / 10,  # Adjust the divisor to control the bubble size
        popup=row['station_name'],
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(map)

# Show the map
map

In [42]:
# sort dfstations to find 10 least frequented stations
dfstations3 = dfstations.sort_values(by='count', ascending=True)
print(dfstations3)

                          neighborhood    borough zipcode  \
stationid                                                   
2001                               NaN   Brooklyn   11251   
2005                               NaN   Brooklyn   11251   
372                       Clinton Hill   Brooklyn   11225   
321                              Dumbo   Brooklyn   11201   
119                                NaN   Brooklyn   11251   
...                                ...        ...     ...   
426        Manhattan Community Board 1  Manhattan   10282   
497        Manhattan Community Board 5  Manhattan   10010   
519        Manhattan Community Board 5  Manhattan   10037   
293        Manhattan Community Board 2  Manhattan   10003   
459        Manhattan Community Board 4  Manhattan   10011   

                            station_name   latitude  longitude  count  
stationid                                                              
2001                 7 Ave & Farragut St  40.698921 -73.973330

In [43]:
df_last_10 = dfstations3.head(10)
print(df_last_10)

                 neighborhood   borough zipcode  \
stationid                                         
2001                      NaN  Brooklyn   11251   
2005                      NaN  Brooklyn   11251   
372              Clinton Hill  Brooklyn   11225   
321                     Dumbo  Brooklyn   11201   
119                       NaN  Brooklyn   11251   
421                       NaN  Brooklyn   11205   
431                       NaN  Brooklyn   11201   
298                       NaN  Brooklyn   11227   
436                       NaN  Brooklyn   11233   
343        Brooklyn Navy Yard  Brooklyn   11251   

                            station_name   latitude  longitude  count  
stationid                                                              
2001                 7 Ave & Farragut St  40.698921 -73.973330      5  
2005              Railroad Ave & Kay Ave  40.705312 -73.971001      5  
372            Franklin Ave & Myrtle Ave  40.694528 -73.958089      8  
321        Cadman Plaza E &

In [44]:
# export last 10
df_last_10.to_csv(os.path.join(path, '02 Data', 'Prepared Data', 'bottom_10.csv'))

In [45]:
bottom_10 = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'bottom_10.csv'), index_col = False)

In [46]:
bottom_10.head()

,stationid,neighborhood,borough,zipcode,station_name,latitude,longitude,count
0,2001,NaN,Brooklyn,11251,7 Ave & Farragut St,40.698921,-73.973330,5
1,2005,NaN,Brooklyn,11251,Railroad Ave & Kay Ave,40.705312,-73.971001,5
2,372,Clinton Hill,Brooklyn,11225,Franklin Ave & Myrtle Ave,40.694528,-73.958089,8
3,321,Dumbo,Brooklyn,11201,Cadman Plaza E & Red Cross Pl,40.699918,-73.989718,11
4,119,NaN,Brooklyn,11251,Park Ave & St Edwards St,40.696089,-73.978034,11


In [47]:
bottom_10.shape

(10, 8)

In [48]:
# Create a Folium map centered around the mean of the coordinates
map = folium.Map(location=[bottom_10['latitude'].mean(), bottom_10['longitude'].mean()], zoom_start=12)
map

In [49]:
# Add CircleMarkers for each city
for index, row in bottom_10.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['count'] / 10,  # Adjust the divisor to control the bubble size
        popup=row['station_name'],
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(map)

# Show the map
map